****************************************************************************
* @file: Behavior.py
* @project: ROBOFEI-HT - FEI 😛
* @author: Vinicius Nicassio Ferreira
* @version: V0.0.1
* @created: 23/10/2017
* @e-mail: vinicius.nicassio@gmail.com
* @brief: Class Behavior
****************************************************************************

In [ ]:
#ini-iPython

# Executando no diretório principal

In [ ]:
import os
os.chdir('../../') #Executando na pasta Visual_Memory
os.system("cd Workbench/ && ./zerar.sh && cd ..")
os.system("cd Workbench/iPython/ && ./convertendo.sh && cd ../../")
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [ ]:
# Libraries to be used.
#des-iPython import sys
#des-iPython sys.path.append('../include')
#des-iPython sys.path.append('../src')

In [ ]:
# The standard libraries used in the visual memory system.
import os # Class used for communication with the system
import signal # Class used for external interrupt detection.
from math import log10, floor # Functions used for rounding UI

In [ ]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe Basic como instanciação
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/Basic.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/Basic.py")
#end-iPython
from Basic import * # Standard and abstract class.
from Robots import * # Class responsible for performing robots tracking.
from Landmark import * # Class responsible for performing landmarks tracking.

# Class Behavior
Class responsible for managing the behavior of Visual Memory.
doxygen-iPython

In [ ]:
#des-iPython class Behavior(Basic):
#ini-tab -> Iniciando código classe

---- Variables ----

# __posrobot
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [ ]:
__posrobot = None #self-iPython __posrobot

# parameters
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [ ]:
parameters = None #self-iPython parameters

In [ ]:
#ini-iPython

# ------- Lista de funções herdadas -------



# ------- Lista variaveis -------

#self-iPython me
#self-iPython land
#self-iPython robots

#self-iPython __newrobots
#self-iPython args

# ------- Lista de funções -------

#self-iPython run
#self-iPython readDataLandmarks
#self-iPython readDataRobots
#self-iPython distributeDataRobots
#self-iPython readDataBall
#self-iPython end
#self-iPython killedProcess
#self-iPython signal_term_handler
#self-iPython printPreviousLine
#self-iPython updatingScreen
#self-iPython roundUI

#end-iPython - Fim listas

In [ ]:
#eself-iPython signal_term_handler
#edes-iPython def signal_term_handler(self, signal, frame):
def signal_term_handler(signal, frame):
    raise VisualMemoryException(3, '') #self-iPython signal_term_handler

In [ ]:
#eself-iPython killedProcess
#edes-iPython def killedProcess(self):
def killedProcess( ):
    signal.signal(signal.SIGTERM, signal_term_handler) #self-iPython killedProcess

# Constructor Class
doxygen-iPython

In [ ]:
#ini-iPython
class argumentos:
    numberrobots = 4
    executionperiod = 2000
    debug = False
    savedata = False
a = argumentos()
#end-iPython
#des-iPython def __init__(self, a):
#ini-tab -> Criando construtor
killedProcess( ) # Function to monitor external process kill.
args = a # Input arguments.

#edes-iPython super(Behavior, self).__init__(a, "Settings", "Visual Memory")
self = Basic(a, "Settings", "Visual Memory")

# Creating default values and reading values
parameters = {
    "number_robots": 8,
    "execution_period_ms": 100,
    "weight_robot": 0.0006,
}
parameters = self._conf.readVariables(parameters)

if args.numberrobots != None:
    parameters["number_robots"] = args.numberrobots

if args.executionperiod != None:
    parameters["execution_period_ms"] = args.executionperiod

if parameters["number_robots"] < 0 or parameters["number_robots"] > 22:
    raise VisualMemoryException(1, parameters["number_robots"])

# Creating objects to be tracking
me = Speeds( )
land = Landmark(args, me)

# Support Variables
robots = []
__newrobots = []
__posrobot = [0, 0, 0]

for i in xrange(-parameters["number_robots"]/2, parameters["number_robots"]/2):
    if i == 0:
        continue
    __newrobots.append(Robots(args, me, __posrobot, i))
a = __newrobots.pop()
#eself-iPython end
print "Finalizando"
a.end() #self-iPython end
print "Finalizado"

In [ ]:
#end-tab -> Finalizando construtor

In [ ]:
#ini-iPython

## Teste construtor

In [ ]:
self._end( )
__newrobots
#end-iPython

# readDataLandmarks
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [ ]:
#eself-iPython readDataLandmarks
#edes-iPython def readDataLandmarks(self, old):
def readDataLandmarks(old):
    if self._bkb.read_int("VISION_LAND_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_LAND_X"), self._bkb.read_float("VISION_LAND_Y")],
                "time": self._bkb.read_double("VISION_LAND_TIME"),
                "movement": self._bkb.read_int("VISION_LAND_MOV"),
            }]
        
        self._bkb.write_int("VISION_LAND_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataLandmarks

In [ ]:
#ini-iPython

## Teste readDataLandmarks

In [ ]:
self._bkb.write_int("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_double("VISION_LAND_TIME", 4)
self._bkb.write_int("VISION_LAND_MOV", 4)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

# readDataRobots
.
doxygen-iPython

In [ ]:
#eself-iPython readDataRobots
#edes-iPython def readDataRobots(self, old):
def readDataRobots(old):
    data = []
    for number in xrange(1, 22):
#         print "number:", number #debug-iPython
        if self._bkb.read_int("VISION_RB" + str(number).zfill(2) + "_TAG") == 0:
            continue
        
        data.append({
            "tag": self._bkb.read_int("VISION_RB" + str(number).zfill(2) + "_TAG") - 2,
            "pos": [self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_X")*100, self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_Y")*100],
            "time": self._bkb.read_double("VISION_RB" + str(number).zfill(2) + "_TIME"),
            "movement": int(self._bkb.read_int("VISION_RB" + str(number).zfill(2) + "_MOV")),
        })
        
        self._bkb.write_int("VISION_RB" + str(number).zfill(2) + "_TAG", 0)
    
    data = sorted(old + data, key= lambda k: k["time"])
    
    return data #self-iPython readDataRobots

## Teste readDataRobots

In [ ]:
#ini-iPython
from IPython.display import clear_output
try:
    while True:
        time.sleep(3)
        clear_output()
        a = []
        a = readDataRobots(a)
        print "a:"
        print np.array(a)
except:
    pass

In [ ]:
self._bkb.write_int("VISION_RB01_TAG", 2)
self._bkb.write_float("VISION_RB01_X", 1)
self._bkb.write_float("VISION_RB01_Y", 2)
self._bkb.write_double("VISION_RB01_TIME", 3)
self._bkb.write_int("VISION_RB01_MOV", 3)

a = []
a = readDataRobots(a)
print "a:", a

self._bkb.write_int("VISION_RB02_TAG", 1)
self._bkb.write_float("VISION_RB02_X", 1)
self._bkb.write_float("VISION_RB02_Y", 2)
self._bkb.write_double("VISION_RB02_TIME", 2)
self._bkb.write_int("VISION_RB02_MOV", 2)

a = readDataRobots(a)
print "a:", a
#end-iPython

# distributeDataRobots
.
doxygen-iPython

In [ ]:
#eself-iPython distributeDataRobots
#edes-iPython def distributeDataRobots(self, datarobots):
def distributeDataRobots(datarobots):
    global __newrobots #debug-iPython
    if datarobots == [] or __newrobots + robots == []:
        return
    
    index = 0
    timenow = -1
    while index < len(datarobots):
        data = datarobots[index]
        print "Dado analisado:", data #debug-iPython
        
        # New instant in time.
        if data["time"] != timenow:
            opponent = [ robot for robot in robots if robot.timenumber < 0 ]
            indefinite = [ robot for robot in robots if robot.timenumber == 0 ]
            teammate = [ robot for robot in robots if robot.timenumber > 1 ]
            print "Criado listas:" #debug-iPython
            print "opponent:", opponent #debug-iPython
            print "indefinite:", indefinite #debug-iPython
            print "teammate:", teammate #debug-iPython
            timenow = data["time"]
        
        # First input data
        if robots == []:
            print "Primeira leitura" #debug-iPython
            candidates = __newrobots.pop(0)
            candidates.timenumber = data["tag"]*(len([ robot for robot in robots if robot.timenumber*data["tag"] > 0 ]) + 1)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            print "\n" #debug-iPython
            raw_input("Continuar...") #debug-iPython
            continue
        
        # Calculates the similarity of the data with the objects
        #  Saving position
        __posrobot[0], __posrobot[1], __posrobot[2] = data["pos"] + [data["time"]]
        
        #  Generating Candidates
        if data["tag"] == -1:
            candidates = opponent + indefinite
        elif data["tag"] == 1:
            candidates = indefinite + teammate
        else:
            candidates = opponent + indefinite + teammate
        print "Candidatos feitos:", candidates #debug-iPython
        
        for cad in candidates:
            print "Pausado antes:", cad._threadPaused(), cad._predictedstate["x"] #debug-iPython
            while not cad._threadPaused():
                time.sleep(0.033333)
            print "Pausado depois:", cad._threadPaused(), cad._predictedstate["x"] #debug-iPython
        
        #  Calculates the similarity
        if len(candidates) == 1:
            print "Apenas 1" #debug-iPython
            candidates[0].calculatesDistance()
        else:
            print "Varios" #debug-iPython
            candidates.sort(reverse=True)
        
        #ini-iPython
        for cad in candidates:
            print "weight:", cad.weight, cad._predictedstate["x"]
        #end-iPython
        
        #ini-iPython
        if candidates != []:
            print "weight:", candidates[0].weight, candidates[0]._predictedstate["x"], candidates[0].weight < parameters["weight_robot"]
        #end-iPython
        
        # Sends the data to the most similar object and run object update
        if __newrobots != [] and (candidates == [] or candidates[0].weight < parameters["weight_robot"]):
            print "Nenhum candidato proximo" #debug-iPython
            candidates = __newrobots.pop(0)
            candidates.timenumber = data["tag"]*(len([ robot for robot in robots if robot.timenumber*data["tag"] > 0 ]) + 1)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            index -= 1
        else:
            if candidates != [] and candidates[0].timenumber < 0:
                print "candidato adversario" #debug-iPython
                opponent.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            elif candidates != [] and candidates[0].timenumber > 0:
                print "cadidato aliado" #debug-iPython
                teammate.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            elif candidates != []:
                print "candidato sei lá" #debug-iPython
                indefinite.remove(candidates[0])
                candidates[0].timenumber = data["tag"]*(len([ robot for robot in robots if robot.timenumber*data["tag"] > 0 ]) + 1)
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
        
        print "Robots:", robots #debug-iPython
        print "New Robots:", __newrobots #debug-iPython
        print "\n" #debug-iPython
        raw_input("Continuar...") #debug-iPython
        index += 1 #self-iPython distributeDataRobots

In [ ]:
#ini-iPython

## Teste distributeDataRobots

In [ ]:
# data = [
#     {"tag": -1, "pos": [+7.00, +0.00], "movement": 1, "time": 0.0},
#     {"tag": 1, "pos": [+6.00, +0.00], "movement": 1, "time": 0.0},
# ]

# data = sorted(data, key= lambda k: k["time"])

# distributeDataRobots(data)

# data = [
#     {"tag": -1, "pos": [+4.49, -1.30], "movement": 1, "time": 0.5},
#     {"tag": 1, "pos": [+6.00, +0.00], "movement": 1, "time": 0.0},
# ]

for i in xrange(1, 22):
    self._bkb.write_int('VISION_RB'+ str(i).zfill(2) +'_TAG', 0)

time.sleep(1)

data = []

data = readDataRobots(data)

distributeDataRobots(data)

print "robots:", robots

#end-iPython

# readDataBall
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [ ]:
#eself-iPython readDataBall
#edes-iPython def readDataBall(self):
def readDataBall(old):
    if self._bkb.read_float("VISION_BALL_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_BALL_X"), self._bkb.read_float("VISION_BALL_Y")],
                "time": self._bkb.read_float("VISION_BALL_TIME"),
                "movement": self._bkb.read_float("VISION_BALL_MOV"),
            }]
        
        self._bkb.write_float("VISION_BALL_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataBall

In [ ]:
#ini-iPython

## Teste readDataLandmarks

In [ ]:
self._bkb.write_int("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_double("VISION_LAND_TIME", 4)
self._bkb.write_int("VISION_LAND_MOV", 5)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

# printPreviousLine
.
doxygen-iPython

In [ ]:
#eself-iPython printPreviousLine
#edes-iPython def printPreviousLine(self, text, lines=1, back=True):
def printPreviousLine(text, lines=1, back=True):
    for i in xrange(lines):
        print "\033[F\r",
    print text

    if back:
        for i in xrange(lines - 1):
            print "" #self-iPython printPreviousLine

In [ ]:
#ini-iPython

## Teste printPreviousLine

In [ ]:
print "Ola mundo"
print "Como vai você?"
printPreviousLine("Batata", lines=3)
#end-iPython

# roundUI
.
doxygen-iPython

In [ ]:
#eself-iPython roundUI
#edes-iPython def roundUI(self, x, digits = 4):
def roundUI(x, digits = 4):
    try:
        return round(x, (digits - 1)-int(floor(log10(abs(x)))))
    except ValueError:
        return 0 #self-iPython roundUI

# updatingScreen
.
doxygen-iPython

In [ ]:
#eself-iPython updatingScreen
#edes-iPython def updatingScreen(self):
def updatingScreen( ):
    # Update landmark
    if self._bkb.read_float("VISUAL_MEMORY_LAND_LOC") == 0:
        text = "+---+ Landmark +     Não    +--------------+           00000          +      "
    else:
        text = "+---+ Landmark +     Sim    +--------------+           00000          +      "
    
    texts = [
        "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_LAND_X"))).zfill(5) +"    |                          |      ",
        text,
        "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_LAND_Y"))).zfill(5) +"    |                          |      ",
    ]
    if parameters["number_robots"] > 1:
        for i, text in zip(range(4 + (parameters["number_robots"] - 1)*4, 4 + (parameters["number_robots"] - 1)*4 - 3, -1), texts):
            printPreviousLine(text, lines=i)
    else:
        for i, text in zip(range(4, 1, -1), texts):
            printPreviousLine(text, lines=i)
    
    # Update robots
    for robot in xrange(1, parameters["number_robots"]):
        # Update friend robots
        if robot <= (parameters["number_robots"] - 1)/2:
            if self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_LOC") == 0:
                text = "+---+  Aliado  +     Não    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "
            else:
                text = "+---+  Aliado  +     Sim    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "

            texts = [
                "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_X"))).zfill(5) +"    |                          |      ",
                text,
                "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_Y"))).zfill(5) +"    |                          |      ",
            ]
        else:
            if self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_LOC") == 0:
                text = "+---+ Oponente +     Não    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "
            else:
                text = "+---+ Oponente +     Sim    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "

            texts = [
                "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_X"))).zfill(5) +"    |                          |      ",
                text,
                "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_Y"))).zfill(5) +"    |                          |      ",
            ]
        
        for i, text in zip(range((parameters["number_robots"] - robot)*4, (parameters["number_robots"] - robot)*4 - 3, -1), texts):
            printPreviousLine(text, lines=i) #self-iPython updatingScreen

In [ ]:
#ini-iPython

## Teste updatingScreen

In [ ]:
updatingScreen( )
#end-iPython

# run
doxygen-iPython

In [ ]:
#eself-iPython run
#edes-iPython def run(self):
def run( ):
    # Initiating variables
    datalandmarks = []
    datarobots = []
    databall = []

    if args.debug:
        print "\nPeríodo de execução:", parameters["execution_period_ms"], "\tPeso mínimo:", parameters["weight_robot"]*100, "%", "\tTempo:", time.strftime("%H:%M:%S", time.localtime(time.time( )))
        print "    +----------+------------+--------------+--------------------------+      "
        print "    |   Nome   | Localizado | Posição (cm) | Velocidade máxima (cm/s) |      "
        print "+---+----------+------------+--------------+--------------------------+      "
        print "| x |          |            |     00000    |                          |      "
        print "+---+ Landmark +     Não    +--------------+           00000          +      "
        print "| y |          |            |     00000    |                          |      "
        print "+---+----------+------------+--------------+--------------------------+      "

        for i in xrange((parameters["number_robots"] - 1)/2):
            print "| x |          |            |     00000    |                          |      "
            print "+---+  Aliado  +     Não    +--------------+           00000          +      "
            print "| y |          |            |     00000    |                          |      "
            print "+---+----------+------------+--------------+--------------------------+      "
        for i in xrange((parameters["number_robots"] - 1)/2, (parameters["number_robots"] - 1)):
            print "| x |          |            |     00000    |                          |      "
            print "+---+ Oponente +     Não    +--------------+           00000          +      "
            print "| y |          |            |     00000    |                          |      "
            print "+---+----------+------------+--------------+--------------------------+      "

    #des-iPython while True:
    #ini-tab - Função while
    try:
        # Start counting time
        start = time.time()
        print "start:", start #debug-iPython

        # Analyzing landmarks ------------------------------------------------------------
        
#         # Reading data from landmarks
#         datalandmarks = readDataLandmarks(datalandmarks)
# #         print "datalandmarks:", datalandmarks

#         if datalandmarks != []:
#             # Predict new landmarks position and robot speed (me)
#             me.update(land.update(datalandmarks.pop(0)))
#         else:
#             # Predicts only the new landmarks position
#             print "Land:" #debug-iPython
#             land.predict(movements = 0)
            
        # Analyzing robots ------------------------------------------------------------

        # Reading robots data
        datarobots = readDataRobots(datarobots)
#         print "datarobots:", datarobots
        
        print "robots:", robots #debug-iPython
        print "newrobots: ", __newrobots #debug-iPython

        # Distribute the data to the robot objects
        print "Recebidos datarobots:", datarobots #debug-iPython
        distributeDataRobots(datarobots)
        print "Restantes datarobots:", datarobots #debug-iPython
        
        print "Robots:", robots #debug-iPython
        print "New Robots: ", __newrobots #debug-iPython

        # # Reading ball data
        # databall = readDataBall(databall)

        # # Distribute the data to the ball objects
        # Batata

        # # Doing cleaning and objects (lost objects)
        i = 0
        while i < len(robots):
            if robots[i].testReset( ):
                __newrobots.append(robots.pop(i))
                continue
            i += 1
                

        # # Predicts objects (Now)
        for robot in robots:
            print "Robots:" #debug-iPython
            robot.predictThread(movements=0)
        #ini-iPython
            while not robot._threadPaused():
                time.sleep(0.033333)
            print "Predito:", robot._state
        raw_input("Novo Ciclo...")
        #end-iPython

        # # Send object for vision screening
        # Batata

        # Displaying debug values
        if args.debug:        
            printPreviousLine(
                "Período de execução: " + str(parameters["execution_period_ms"]) + "\tPeso mínimo: " + str(parameters["weight_robot"]*100) + "%\tTempo: " + time.strftime("%H:%M:%S", time.localtime(time.time( ))),
                lines=8+4*(parameters["number_robots"]-1)
            )            
            updatingScreen( )

        # Wait an instant on the remaining time
        delta = parameters["execution_period_ms"]*1e-3 - time.time() + start
        print "delta:", delta #debug-iPython
        if delta > 0:
            print "Aguardando", delta*1e3, "ms" #debug-iPython
            time.sleep(delta) #self-iPython run
            print "Período finalizado" #debug-iPython
    except KeyboardInterrupt:
        os.system('clear') # Cleaning terminal
        print "Keyboard interrupt detected"
        #edes-iPython break
        pass
    except VisualMemoryException as e:
        #edes-iPython break
        pass
    #end-tab - Função while

In [ ]:
#ini-iPython

## Teste run

In [ ]:
run( )
#end-iPython

# end
.
doxygen-iPython

In [ ]:
#eself-iPython end
#edes-iPython def end(self):
def end( ):
    self._end( )
    kill = robots + __newrobots
    print "Lista de robos:", kill #debug-iPython
    
    for robot in kill:
        print "Finalizando", robot #debug-iPython
        robot.end( )
        print "Finalizado", robot #debug-iPython
    
    land.end( ) #self-iPython end

In [ ]:
#ini-iPython

## Teste end

In [ ]:
end( )
#end-iPython

In [ ]:
#end-tab -> Finalizando classe

In [ ]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [ ]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [ ]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [ ]:
#ini-iPython

## Teste nova_funcao

In [ ]:

#end-iPython

In [ ]:
#end-iPython